In [1]:
import torch.nn as nn

from datasets import *
from evaluation import *
from models import *
from pruning.unstructured import *
from pruning.structured import *

Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/unstructured/L1norm.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/Train.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/unstructured/random.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/decay_pruning.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/two_to_four.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/Thinet.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/Global_pruning.py
Importing /raid/home/prabhasreddy/sysforml/Smart-pruner-dev/pruning/structured/Ln_structured.py


In [2]:
dataset = MNIST()
train_loader, test_loader = dataset.get_dataloader()

In [3]:
#train model and return it
model = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = Trainer(model, 5, train_loader, criterion, optimizer)
original_model = model.train()
#print("Accuracy of original model : ", accuracy(original_model, test_loader))

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 5.0579 | 
Epoch: 2 | train_loss: 3.5774 | 
Epoch: 3 | train_loss: 3.2134 | 
Epoch: 4 | train_loss: 2.9422 | 
Epoch: 5 | train_loss: 2.6897 | 


In [5]:
#for UnstructuredL1normPrune
# model = Lenet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(original_model.parameters(), lr=0.001)

trainer = GlobalPrune(original_model, 5, train_loader, criterion, optimizer, 0.9)

#train and prune the model and return both the original and pruned model
#original_model, pruned_model = trainer.train_prune_retrain()
pruned_model = trainer.prune_model()

In [6]:
%%bash --bg 
watch -n 1 'nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> unpruned_output.txt'

In [7]:
print("Accuracy of original model : ", accuracy(original_model, test_loader))

with open('unpruned_output.txt', 'a') as f:
    f.write("\energy computation for original model completed.\n")

Accuracy of original model :  [98.62]


In [8]:
%%bash
pgrep -f "watch -n 1 'nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> unpruned_output.txt'" | xargs -r kill

In [9]:
#for pruned 

In [10]:
%%bash --bg 
watch -n 1 'nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> pruned_output.txt'

In [11]:
print("Accuracy of pruned model : ", accuracy(pruned_model, test_loader))

with open('pruned_output.txt', 'a') as f:
    f.write("\energy computation for pruned model completed.\n")

Accuracy of pruned model :  [66.19]


In [12]:
%%bash
pgrep -f "watch -n 1 'nvidia-smi -i 0 --format=csv,noheader,nounits --query-gpu=power.draw >> pruned_output.txt'" | xargs -r kill

In [13]:
#done

In [14]:
total = 0
count=0
with open('unpruned_output.txt', 'r') as f:
    for line in f:
        if '\\energy' in line:
            break
        total += float(line.strip())
        count += 1

average = total / count if count else 0
print(average)

56.59


In [15]:
total = 0
count =0 
with open('pruned_output.txt', 'r') as f:
    for line in f:
        if '\\energy' in line:
            break
        total += float(line.strip()) 
        count += 1

average = total / count if count else 0
print(average)

56.59
